# Singapore Job Market - Salary Analysis & Market Benchmarking

**Client:** HR Consulting Firm  
**Objective:** Benchmark salaries for different roles and skills in the Singapore job market

## Table of Contents
1. [Data Loading & Initial Exploration](#1-data-loading)
2. [Data Cleaning & Preprocessing](#2-data-cleaning)
3. [Exploratory Data Analysis](#3-exploratory-data-analysis)
4. [Salary Analysis by Role & Experience](#4-salary-analysis)
5. [Correlation Analysis](#5-correlation-analysis)
6. [Outlier Detection](#6-outlier-detection)
7. [Key Insights & Recommendations](#7-insights)

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

## 1. Data Loading & Initial Exploration <a id='1-data-loading'></a>

In [ ]:
# Load the dataset
df = pd.read_csv('../data/SGJobData.csv', encoding='utf-8-sig')

print("Dataset Shape:", df.shape)
print("\nFirst few rows:")
df.head()

In [ ]:
# Dataset info
print("Dataset Information:")
df.info()

In [ ]:
# Check for missing values
missing_data = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2)
})
missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)
print("\nMissing Values Summary:")
missing_data

In [ ]:
# Basic statistics
print("\nBasic Statistics:")
df.describe()

## 2. Data Cleaning & Preprocessing <a id='2-data-cleaning'></a>

In [ ]:
# Create a copy for cleaning
df_clean = df.copy()

# Parse categories from JSON string to extract primary category
def extract_primary_category(categories_str):
    try:
        if pd.isna(categories_str):
            return 'Unknown'
        categories = json.loads(categories_str)
        if categories and len(categories) > 0:
            return categories[0]['category']
        return 'Unknown'
    except (json.JSONDecodeError, TypeError, KeyError, IndexError) as e:
        # Log warning for debugging purposes
        # print(f"Warning: Failed to parse categories: {e}")
        return 'Unknown'

df_clean['primary_category'] = df_clean['categories'].apply(extract_primary_category)

# Convert date columns to datetime
date_columns = ['metadata_expiryDate', 'metadata_newPostingDate', 'metadata_originalPostingDate']
for col in date_columns:
    df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')

# Calculate average salary
df_clean['salary_average'] = (df_clean['salary_minimum'] + df_clean['salary_maximum']) / 2

# Calculate salary range
df_clean['salary_range'] = df_clean['salary_maximum'] - df_clean['salary_minimum']

# Fill missing position levels
df_clean['positionLevels'] = df_clean['positionLevels'].fillna('Not Specified')

# Fill missing years of experience with 0
df_clean['minimumYearsExperience'] = df_clean['minimumYearsExperience'].fillna(0)

# Filter only monthly salaries for consistency
df_monthly = df_clean[df_clean['salary_type'] == 'Monthly'].copy()

print(f"Original dataset: {len(df)} rows")
print(f"After filtering monthly salaries: {len(df_monthly)} rows")
print(f"Rows removed: {len(df) - len(df_monthly)}")

In [ ]:
# Remove extreme outliers (salaries that are unrealistic)
# Keep salaries between $1000 and $50000 per month
df_monthly = df_monthly[
    (df_monthly['salary_minimum'] >= 1000) & 
    (df_monthly['salary_maximum'] <= 50000)
].copy()

print(f"After removing extreme salary outliers: {len(df_monthly)} rows")

In [ ]:
# Check cleaned data
print("\nCleaned Data Sample:")
df_monthly[['title', 'primary_category', 'positionLevels', 'minimumYearsExperience', 
            'salary_minimum', 'salary_maximum', 'salary_average']].head(10)

## 3. Exploratory Data Analysis <a id='3-exploratory-data-analysis'></a>

In [ ]:
# Overall salary distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Histogram of average salaries
axes[0, 0].hist(df_monthly['salary_average'], bins=50, color='skyblue', edgecolor='black')
axes[0, 0].set_xlabel('Average Salary (SGD)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Distribution of Average Salaries')
axes[0, 0].axvline(df_monthly['salary_average'].median(), color='red', linestyle='--', label=f'Median: ${df_monthly["salary_average"].median():.0f}')
axes[0, 0].legend()

# Box plot of average salaries
axes[0, 1].boxplot(df_monthly['salary_average'], vert=True)
axes[0, 1].set_ylabel('Average Salary (SGD)')
axes[0, 1].set_title('Box Plot of Average Salaries')

# Distribution of minimum salaries
axes[1, 0].hist(df_monthly['salary_minimum'], bins=50, color='lightcoral', edgecolor='black')
axes[1, 0].set_xlabel('Minimum Salary (SGD)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Distribution of Minimum Salaries')

# Distribution of maximum salaries
axes[1, 1].hist(df_monthly['salary_maximum'], bins=50, color='lightgreen', edgecolor='black')
axes[1, 1].set_xlabel('Maximum Salary (SGD)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Distribution of Maximum Salaries')

plt.tight_layout()
plt.show()

# Print summary statistics
print("\nSalary Summary Statistics:")
print(df_monthly[['salary_minimum', 'salary_maximum', 'salary_average']].describe())

In [ ]:
# Top job categories by count
top_categories = df_monthly['primary_category'].value_counts().head(15)

plt.figure(figsize=(12, 6))
top_categories.plot(kind='barh', color='steelblue')
plt.xlabel('Number of Job Postings')
plt.ylabel('Job Category')
plt.title('Top 15 Job Categories by Number of Postings')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print(f"\nTotal unique job categories: {df_monthly['primary_category'].nunique()}")

In [ ]:
# Position levels distribution
position_counts = df_monthly['positionLevels'].value_counts()

plt.figure(figsize=(10, 6))
position_counts.plot(kind='bar', color='coral')
plt.xlabel('Position Level')
plt.ylabel('Number of Job Postings')
plt.title('Distribution of Job Postings by Position Level')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 4. Salary Analysis by Role & Experience <a id='4-salary-analysis'></a>

In [ ]:
# Average salary by position level
salary_by_position = df_monthly.groupby('positionLevels').agg({
    'salary_average': ['mean', 'median', 'std'],
    'title': 'count'
}).round(2)

salary_by_position.columns = ['Mean_Salary', 'Median_Salary', 'Std_Dev', 'Job_Count']
salary_by_position = salary_by_position.sort_values('Median_Salary', ascending=False)

print("\nSalary Statistics by Position Level:")
print(salary_by_position)

In [ ]:
# Visualize salary by position level
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar plot of median salaries
salary_by_position['Median_Salary'].plot(kind='barh', ax=axes[0], color='teal')
axes[0].set_xlabel('Median Salary (SGD)')
axes[0].set_ylabel('Position Level')
axes[0].set_title('Median Salary by Position Level')
axes[0].invert_yaxis()

# Box plot by position level
position_order = salary_by_position.index.tolist()
df_monthly_sorted = df_monthly[df_monthly['positionLevels'].isin(position_order)]
sns.boxplot(data=df_monthly_sorted, y='positionLevels', x='salary_average', 
            order=position_order, ax=axes[1])
axes[1].set_xlabel('Average Salary (SGD)')
axes[1].set_ylabel('Position Level')
axes[1].set_title('Salary Distribution by Position Level')

plt.tight_layout()
plt.show()

In [ ]:
# Average salary by job category (top 10)
salary_by_category = df_monthly.groupby('primary_category').agg({
    'salary_average': ['mean', 'median'],
    'title': 'count'
}).round(2)

salary_by_category.columns = ['Mean_Salary', 'Median_Salary', 'Job_Count']
salary_by_category = salary_by_category[salary_by_category['Job_Count'] >= 10]  # Filter categories with at least 10 jobs
salary_by_category = salary_by_category.sort_values('Median_Salary', ascending=False).head(15)

print("\nTop 15 Highest Paying Job Categories (with at least 10 postings):")
print(salary_by_category)

In [ ]:
# Visualize top paying categories
plt.figure(figsize=(12, 8))
salary_by_category['Median_Salary'].plot(kind='barh', color='goldenrod')
plt.xlabel('Median Salary (SGD)')
plt.ylabel('Job Category')
plt.title('Top 15 Highest Paying Job Categories (Min 10 Postings)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Salary by years of experience
salary_by_experience = df_monthly.groupby('minimumYearsExperience').agg({
    'salary_average': ['mean', 'median'],
    'title': 'count'
}).round(2)

salary_by_experience.columns = ['Mean_Salary', 'Median_Salary', 'Job_Count']
salary_by_experience = salary_by_experience[salary_by_experience.index <= 20]  # Focus on 0-20 years

print("\nSalary by Years of Experience:")
print(salary_by_experience)

In [ ]:
# Visualize salary vs experience
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Line plot
axes[0].plot(salary_by_experience.index, salary_by_experience['Mean_Salary'], 
             marker='o', label='Mean Salary', linewidth=2)
axes[0].plot(salary_by_experience.index, salary_by_experience['Median_Salary'], 
             marker='s', label='Median Salary', linewidth=2)
axes[0].set_xlabel('Minimum Years of Experience')
axes[0].set_ylabel('Salary (SGD)')
axes[0].set_title('Salary Trend by Years of Experience')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Scatter plot with trend line
axes[1].scatter(df_monthly['minimumYearsExperience'], df_monthly['salary_average'], 
                alpha=0.3, s=20)
z = np.polyfit(df_monthly['minimumYearsExperience'], df_monthly['salary_average'], 1)
p = np.poly1d(z)
axes[1].plot(df_monthly['minimumYearsExperience'], 
             p(df_monthly['minimumYearsExperience']), 
             "r--", linewidth=2, label='Trend Line')
axes[1].set_xlabel('Minimum Years of Experience')
axes[1].set_ylabel('Average Salary (SGD)')
axes[1].set_title('Salary vs Experience (Scatter Plot)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Correlation Analysis <a id='5-correlation-analysis'></a>

In [ ]:
# Select numeric columns for correlation analysis
numeric_cols = ['minimumYearsExperience', 'numberOfVacancies', 'salary_minimum', 
                'salary_maximum', 'salary_average', 'salary_range',
                'metadata_totalNumberJobApplication', 'metadata_totalNumberOfView']

correlation_matrix = df_monthly[numeric_cols].corr()

# Plot correlation heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1)
plt.title('Correlation Matrix of Key Variables')
plt.tight_layout()
plt.show()

print("\nKey Correlations with Salary Average:")
salary_corr = correlation_matrix['salary_average'].sort_values(ascending=False)
print(salary_corr)

In [ ]:
# Analyze salary range patterns
plt.figure(figsize=(12, 6))
plt.scatter(df_monthly['salary_average'], df_monthly['salary_range'], alpha=0.4)
plt.xlabel('Average Salary (SGD)')
plt.ylabel('Salary Range (Max - Min)')
plt.title('Salary Range vs Average Salary')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate average salary range percentage
df_monthly['range_percentage'] = (df_monthly['salary_range'] / df_monthly['salary_average'] * 100)
print(f"\nAverage salary range as % of average salary: {df_monthly['range_percentage'].mean():.1f}%")
print(f"Median salary range as % of average salary: {df_monthly['range_percentage'].median():.1f}%")

## 6. Outlier Detection <a id='6-outlier-detection'></a>

In [ ]:
# Identify high-paying outliers using IQR method
Q1 = df_monthly['salary_average'].quantile(0.25)
Q3 = df_monthly['salary_average'].quantile(0.75)
IQR = Q3 - Q1

outlier_threshold_high = Q3 + 1.5 * IQR
outlier_threshold_low = Q1 - 1.5 * IQR

high_salary_outliers = df_monthly[df_monthly['salary_average'] > outlier_threshold_high]

print(f"\nOutlier Detection (IQR Method):")
print(f"Lower threshold: ${outlier_threshold_low:.2f}")
print(f"Upper threshold: ${outlier_threshold_high:.2f}")
print(f"Number of high-salary outliers: {len(high_salary_outliers)}")
print(f"Percentage of dataset: {len(high_salary_outliers)/len(df_monthly)*100:.2f}%")

In [ ]:
# Examine high-paying outliers
print("\nTop 10 Highest Paying Jobs:")
top_paying_jobs = df_monthly.nlargest(10, 'salary_average')[[
    'title', 'primary_category', 'positionLevels', 'minimumYearsExperience',
    'salary_minimum', 'salary_maximum', 'salary_average', 'postedCompany_name'
]]
print(top_paying_jobs.to_string())

In [ ]:
# Analyze categories with highest salary variance
category_variance = df_monthly.groupby('primary_category').agg({
    'salary_average': ['mean', 'std', 'count']
}).round(2)

category_variance.columns = ['Mean_Salary', 'Std_Dev', 'Count']
category_variance = category_variance[category_variance['Count'] >= 10]
category_variance['CV'] = (category_variance['Std_Dev'] / category_variance['Mean_Salary'] * 100).round(2)
category_variance = category_variance.sort_values('Std_Dev', ascending=False).head(10)

print("\nTop 10 Job Categories with Highest Salary Variance:")
print(category_variance)

## 7. Key Insights & Recommendations <a id='7-insights'></a>

In [ ]:
# Generate summary statistics for reporting
print("="*80)
print("SINGAPORE JOB MARKET - SALARY BENCHMARKING SUMMARY")
print("="*80)

print(f"\n1. OVERALL MARKET STATISTICS")
print(f"   - Total job postings analyzed: {len(df_monthly):,}")
print(f"   - Median monthly salary: ${df_monthly['salary_average'].median():,.0f}")
print(f"   - Mean monthly salary: ${df_monthly['salary_average'].mean():,.0f}")
print(f"   - Salary range (25th-75th percentile): ${df_monthly['salary_average'].quantile(0.25):,.0f} - ${df_monthly['salary_average'].quantile(0.75):,.0f}")

print(f"\n2. TOP 3 HIGHEST PAYING CATEGORIES (with sufficient data):")
top_3_categories = salary_by_category.head(3)
for idx, (cat, row) in enumerate(top_3_categories.iterrows(), 1):
    print(f"   {idx}. {cat}: ${row['Median_Salary']:,.0f} median (n={int(row['Job_Count'])} jobs)")

print(f"\n3. POSITION LEVEL INSIGHTS:")
top_3_positions = salary_by_position.head(3)
for idx, (pos, row) in enumerate(top_3_positions.iterrows(), 1):
    print(f"   {idx}. {pos}: ${row['Median_Salary']:,.0f} median (n={int(row['Job_Count'])} jobs)")

print(f"\n4. EXPERIENCE VS SALARY:")
corr_exp_salary = df_monthly['minimumYearsExperience'].corr(df_monthly['salary_average'])
print(f"   - Correlation between experience and salary: {corr_exp_salary:.3f}")

entry_level_median = df_monthly[df_monthly['minimumYearsExperience'] == 0]['salary_average'].median()
exp_5_median = df_monthly[df_monthly['minimumYearsExperience'] == 5]['salary_average'].median()
exp_10_median = df_monthly[df_monthly['minimumYearsExperience'] == 10]['salary_average'].median()

print(f"   - Entry level (0 yrs): ${entry_level_median:,.0f} median")
print(f"   - Mid-level (5 yrs): ${exp_5_median:,.0f} median")
print(f"   - Senior (10 yrs): ${exp_10_median:,.0f} median")

print(f"\n5. MARKET DEMAND:")
print(f"   - Most in-demand category: {top_categories.index[0]} ({top_categories.iloc[0]} postings)")
print(f"   - Most common position level: {position_counts.index[0]} ({position_counts.iloc[0]} postings)")

print("\n" + "="*80)

### Key Recommendations for HR Consulting Firm:

1. **Market Positioning**: Use the median salary benchmarks by category and position level to advise clients on competitive compensation packages.

2. **Experience Premium**: There is a clear correlation between years of experience and salary. Recommend structured progression paths.

3. **Category Focus**: High-paying categories identified should be prioritized for talent acquisition strategies.

4. **Salary Range Strategy**: The typical salary range represents flexibility in negotiation. Wider ranges may indicate skill variation within roles.

5. **Outlier Analysis**: High-paying outliers often represent specialized skills or senior leadership positions worth investigating.

6. **Regular Updates**: Market dynamics change - recommend quarterly updates to these benchmarks.

In [ ]:
# Save cleaned data for dashboard
df_monthly.to_csv('../data/cleaned_salary_data.csv', index=False)
print("\nCleaned data saved to '../data/cleaned_salary_data.csv'")